# **Chapter 7: Synchronization Examples – Practice Exercise Solutions**

## **7.1 Multiple Locking Mechanisms in Windows and Linux**

**Question:**  
Explain why Windows and Linux implement multiple locking mechanisms. Describe the circumstances under which they use spinlocks, mutex locks, semaphores, and condition variables. In each case, explain why the mechanism is needed.

**Answer:**  
These operating systems provide different locking mechanisms depending on the application developers’ needs.  
- **Spinlocks**: Used in **multiprocessor systems** for **short-duration** critical sections where the cost of context switching (sleep/wake) is higher than briefly spinning. They avoid scheduler overhead.  
- **Mutex locks**: Used for general **mutual exclusion** when a resource may be held for longer periods. They block the thread (put it to sleep) if the lock is unavailable, saving CPU cycles. Some systems (like Solaris) use **adaptive mutexes** that spin briefly before blocking.  
- **Semaphores**: Used for **counting resources** (e.g., limiting concurrent accesses) or synchronizing between threads/processes where a thread may need to wait for a signal from another. Useful for **long waits**.  
- **Condition variables**: Used with mutexes for **complex synchronization patterns** (e.g., producer-consumer) where a thread must wait for a specific condition to become true. They allow efficient waiting without busy polling.

Each mechanism addresses different performance and correctness trade-offs.

---

## **7.2 Slim Reader-Writer Locks in Windows**

**Question:**  
Windows provides a lightweight synchronization tool called slim reader–writer locks. Whereas most implementations of reader–writer locks favor either readers or writers, or perhaps order waiting threads using a FIFO policy, slim reader–writer locks favor neither readers nor writers, nor are waiting threads ordered in a FIFO queue. Explain the benefits of providing such a synchronization tool.

**Answer:**  
**Simplicity and speed**. Slim reader-writer locks avoid the overhead of fairness policies (like FIFO ordering) or favoring readers/writers, which require additional queue management and state tracking. This makes acquisition and release **faster** with lower memory overhead. They are appropriate when strict fairness is not required, but fast, lightweight read-write synchronization is needed—common in high-performance kernel or user-space code where lock contention is moderate.

---

## **7.3 Replacing Binary Semaphore with Mutex Lock**

**Question:**  
Describe what changes would be necessary to the producer and consumer processes in Figure 7.1 and Figure 7.2 so that a mutex lock could be used instead of a binary semaphore.

**Answer:**  
The calls to `wait(mutex)` and `signal(mutex)` (where `mutex` is a binary semaphore) need to be replaced with the **mutex lock API** calls:  
- `wait(mutex)` → `acquire(mutex)` or `lock(mutex)`  
- `signal(mutex)` → `release(mutex)` or `unlock(mutex)`  

The logic remains identical; only the synchronization primitive changes. The mutex lock provides the same mutual exclusion but is often implemented with support for ownership (to prevent a thread from unlocking a mutex it doesn’t hold) and may include priority inheritance features.

---

## **7.4 Deadlock in Dining Philosophers Problem**

**Question:**  
Describe how deadlock is possible with the dining-philosophers problem.

**Answer:**  
If all philosophers simultaneously pick up their **left fork**, each will hold one fork and wait indefinitely for their **right fork**, which is held by the neighbor to their right. Since no philosopher puts down a fork until they eat, all are blocked forever—a **circular wait** condition leading to deadlock.

---

## **7.5 Signaled vs. Non-Signaled States in Windows**

**Question:**  
Explain the difference between signaled and non-signaled states with Windows dispatcher objects.

**Answer:**  
- **Signaled state**: The object is **available**. A thread attempting to acquire it (e.g., via `WaitForSingleObject`) will **not block** and will proceed immediately. The object may represent a lock that is free, an event that has occurred, etc.  
- **Non-signaled state**: The object is **unavailable**. A thread trying to acquire it will **block** until the object transitions to signaled (e.g., when a lock is released or an event is set).

---

## **7.6 Atomic Operations in Linux**

**Question:**  
Assume `val` is an atomic integer in a Linux system. What is the value of `val` after the following operations have been completed?  
```
atomic_set(&val,10);
atomic_sub(8,&val);
atomic_inc(&val);
atomic_inc(&val);
atomic_add(6,&val);
atomic_sub(3,&val);
```

**Answer:**  
Calculating sequentially:  
1. `atomic_set(&val,10)` → val = 10  
2. `atomic_sub(8,&val)` → val = 10 - 8 = 2  
3. `atomic_inc(&val)` → val = 2 + 1 = 3  
4. `atomic_inc(&val)` → val = 3 + 1 = 4  
5. `atomic_add(6,&val)` → val = 4 + 6 = 10  
6. `atomic_sub(3,&val)` → val = 10 - 3 = 7  

**Final value of val is 7**.
